In [1]:
import cv2, os, sys, argparse
import numpy as np
from PIL import Image
from skimage import io
from sklearn.cross_validation import train_test_split
from keras.models import Sequential, model_from_json
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout  
from keras.utils import np_utils
from matplotlib import pyplot as plt
%matplotlib inline

C:\Users\Luigi Ocampo\Anaconda3\envs\thesis\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Luigi Ocampo\Anaconda3\envs\thesis\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
face_data = []
face_labels = []
current_label = 0
id_label = {}

In [3]:
image_dir = "C:\\Users\\Luigi Ocampo\\Documents\\DLSU\\Thesis\\Data\\RGB"

In [4]:
for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith("jpg"):
                path = os.path.join(root,file)
                label = os.path.basename(os.path.dirname(path))
                if not label in id_label:
                    id_label[label] = current_label
                    current_label += 1
                ids = id_label[label]
                pil_image = Image.open(path).convert("L")
                pil_image = pil_image.resize((150,150))
                data_array = np.array(pil_image, "uint8")
                face_data.append(data_array)
                face_labels.append(ids)

In [5]:
Xtr, Xte, ytr, yte = train_test_split(np.array(face_data),np.array(face_labels), train_size=0.9, random_state=1983)

In [6]:
Xtr = np.array(Xtr)
Xte = np.array(Xte)
print(Xtr.shape)
print(Xte.shape)

(28800, 150, 150)
(3200, 150, 150)


In [7]:
classes = 4

In [8]:
ytr = np.array(ytr)
yte = np.array(yte)
Ytr = np_utils.to_categorical(ytr, classes)
Yte = np_utils.to_categorical(yte, classes)

In [9]:
Xtr = Xtr.reshape(28800, 150*150)
Xte = Xte.reshape(3200, 150*150)

In [10]:
Xtr = Xtr.astype('float32')
Xte = Xte.astype('float32')

Xtr /= 255
Xte /= 255

In [11]:
print("Training matrix shape", Xtr.shape)
print("Testing matrix shape", Xte.shape)

Training matrix shape (28800, 22500)
Testing matrix shape (3200, 22500)


In [17]:
model = Sequential()
model.add(Dense(512,input_shape=(Xtr.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(classes))
model.add(Activation('softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [19]:
model.fit(Xtr, Ytr, batch_size=128, epochs=200, verbose=1, validation_data=(Xte, Yte))

Train on 28800 samples, validate on 3200 samples
Epoch 1/200
28800/28800 [==============================] - 63s 2ms/step - loss: 8.2067 - acc: 0.4785 - val_loss: 8.2504 - val_acc: 0.4825
Epoch 2/200
 5632/28800 [====>.........................] - ETA: 31s - loss: 8.1564 - acc: 0.4924

KeyboardInterrupt: 

In [30]:
model_json = model.to_json()
with open("thesis_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("thesis_model.h5")

In [ ]:
score = model.evaluate(Xte, Yte, verbose=1)
print("Test score:", score[0])
print('Test accuracy:', score[1])

In [ ]:
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()